# Generate object(sign) and annotations on draw

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os, sys
from glob import glob
from matplotlib import pyplot

from PIL import Image
from PIL import EpsImagePlugin
import math

from PIL import ImageFilter
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2


# preprocessing (Test)

In [93]:
img = cv2.imread('DT_177.png',-1)
kernel = np.ones((10, 10), np.uint8)
img = cv2.erode(img, kernel, iterations = 1)
    
cv2.imwrite(s, img) 

True

In [98]:
im = cv2.imread('./data/dataset/1-f_pan.png',cv2.IMREAD_GRAYSCALE)
cv2.imwrite('./data/Gray.png', im)

ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)
cv2.imwrite('./data/testGray.png', imGray)

True

In [20]:
img =  cv2.imread('./DT_150.png')


kernel = np.ones((20, 20), np.uint8)
result4 = cv2.erode(img, kernel, iterations = 1)

kernel = np.ones((30, 30), np.uint8)
result1 = cv2.erode(img, kernel, iterations = 1)

cv2.imwrite('./TTEST_erode20.png',  result4) 
cv2.imwrite('./TTEST_erode30.png',  result1) 

True

In [105]:
img =  cv2.imread('./DT_150.png')

kernel = np.ones((7, 7), np.uint8)
result = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


cv2.imwrite('./TTEST_dilate.png',  result ) 

True

----------------------------------

# Preprocessing

In [109]:
sign_data = glob('./data/backGround/sign/*.png')
# preprocessing (Sign)

for s in sign_data:
    img = cv2.imread(s)
    kernel = np.ones((10, 10), np.uint8)
    img = cv2.erode(img, kernel, iterations = 1)
    
    cv2.imwrite(s, img)  
    

In [110]:
size = 20, 20

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im = im.convert("RGBA")
    datas = im.getdata()
    newData = []
    for item in datas:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)

    im.putdata(newData)
    im.save(s)

In [105]:
size = 20, 20

im = Image.open('./data/backGround/sign/DT_177.png')
im.thumbnail(size, Image.ANTIALIAS)
im = im.convert("RGBA")
datas = im.getdata()
newData = []
for item in datas:
    if item[0] > 200 and item[1] > 200 and item[2] > 200:
        newData.append((item[0], item[1], item[2], 0))
    else:
        newData.append(item)

im.putdata(newData)
im.save(s)

# Change color of sign

In [96]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = Image.open(inputImg)
    
    im = np.array(im)
    ret, im = cv2.threshold(im, 247, 255, cv2.THRESH_TOZERO)
    data = im
    
    
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)    
    imDir = './data/backGround/New/sign/'+imName+'_r'+str(r2)+'g'+str(g2)+'b'+str(b2)+'.png'
    im.save(imDir)
    
    return imDir

#  Make n-Sign on Draw with loaction(x,y) information 

In [97]:
sign_data = glob('./data/backGround/sign/*.png')
sign_data 

['./data/backGround/sign/DT_002.png',
 './data/backGround/sign/DT_001.png',
 './data/backGround/sign/SM_142.png',
 './data/backGround/sign/DT_143.png',
 './data/backGround/sign/SM_126.png',
 './data/backGround/sign/SM_100.png',
 './data/backGround/sign/DT_150.png',
 './data/backGround/sign/MD_019.png',
 './data/backGround/sign/DT_131.png',
 './data/backGround/sign/DT_127.png',
 './data/backGround/sign/DT_177.png']

In [101]:
draw_data = glob('./data/backGround/draw/*.png')

for i in draw_data:
    im = cv2.imread(i)
    ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)

    gray = cv2.cvtColor(imGray, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(i, gray)

In [111]:
dr_fileName = './data/backGround/draw/10f.png'

l_img =  cv2.imread(dr_fileName)
h,w,d = l_img.shape
    
sign_data = glob('./data/backGround/sign/*.png')

signList=[]
# set a color RGB
r2,g2,b2 = 255, 0, 0

# set an amoumnt of sign #

DT_002_amount = 2
DT_001_amount = 2
SM_142_amount = 2
DT_143_amount = 2
SM_126_amount = 2
SM_100_amount = 2
DT_150_amount = 2
MD_019_amount = 2
DT_131_amount = 2
DT_127_amount = 2
DT_177_amount = 2


signAmount = [DT_002_amount,DT_001_amount, SM_142_amount, DT_143_amount, SM_126_amount,SM_100_amount, DT_150_amount, MD_019_amount, DT_131_amount, DT_127_amount, DT_177_amount]
total = sum(signAmount)


new_fileName  =  'GEN_'+str(int(random.random()*100000))+'.png'
new_dirFileName = './data/backGround/New/draw/'+ new_fileName


root = etree.Element("annotation")

folder = etree.SubElement(root, "folder").text ='draw'  
filename =  etree.SubElement(root, "filename").text = new_fileName    
path =  etree.SubElement(root, "path").text = new_dirFileName

source =  etree.SubElement(root, "source")
database =  etree.SubElement(source, "database").text= 'Unknown'     
    
size =  etree.SubElement(root, "size")
width =  etree.SubElement(size, "width").text = str(w)
height =  etree.SubElement(size, "height").text = str(h)
depth =  etree.SubElement(size, "depth").text = str(d)

segmented =  etree.SubElement(root, "segmented").text = '0'


print(total) # 생성할 기호의 총합
print(new_dirFileName) #생성될 도면의 경로


xmin = random.sample(range(50, w), total)
ymin = random.sample(range(50, h), total)
    
location = list(zip(xmin,ymin))


for i, j in zip(sign_data , signAmount):
    
    signDir = i
    clssName = signDir.split('/sign')[1]
    clssName = clssName.split('.png')[0]
    
    signList.append(clssName)

    imDir = ChangeColor(signDir,r2,g2,b2)
      
    s_img =  cv2.imread(imDir,-1)     
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s 
        
            
    for k in range(0,j):
    
        x1, y1 = location.pop()
        x2 = x1 + s_img.shape[1]
        y2 = y1 + s_img.shape[0]
               
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clssName
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'

        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])
            
cv2.imwrite(new_dirFileName, l_img)        
tree = etree.ElementTree(root)
 
with open('./data/backGround/annotations/'+'GEN_'+str(int(random.random()*100000))+'xml', "bw") as fh:
    tree.write(fh)
    
      

22
./data/backGround/New/draw/GEN_33384.png
230 250 727 747
1362 1382 708 728
1352 1372 430 450
379 399 853 873
1378 1398 801 816
517 537 473 488
1240 1260 780 800
360 380 899 919
1128 1148 954 974
194 214 366 386
1290 1310 646 657
288 308 331 342
168 188 299 310
68 88 737 748
564 584 764 778
948 968 772 786
1007 1027 327 347
938 958 309 329
529 549 200 220
747 767 505 525
131 151 683 694
1116 1136 193 204
